# Mapping the Signal Beacons from Aeschylus' Agamemnon
In Aeschylus' "Agamemnon," Clytaemnestra describes the sequence of beacons which sent the signal of Troy's defeat from Troy all the way to Argos. In this lesson, we will use Natural Language Processing and Named Entity Recognition to find the locations within this text, then use Pleiades and ArcGIS to map the locations and trace the path which the signals are said to have taken to evaluate whether or not it makes sense. 

## Installing spaCy
Our first step is to install spaCy, the Python library for NLP. After that, we will install spaCy's medium model for Named Entity Recognition. In our case, the medium model should be sufficient, as the large model proved to be no more effective. 

In [1]:
%pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 54.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 56.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 41.3 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [9]:
%run -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 57.8 MB/s eta 0:00:0000:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## Run NER
Now it's time to run the Named Entity Recognition. We can simply copy and paste the text here as a string, since it is short enough as to make creating and opening a .txt document superfluous. 

In [10]:
import spacy

In [11]:
# Load the model that we downloaded.
# If this line fails, make sure that
# you have downloaded the model that's
# referenced here.
nlp = spacy.load("en_core_web_md")

doc = nlp("""
Clytaemestra
Hephaestus, from Ida speeding forth his brilliant blaze. Beacon passed beacon on to
us by courier-flame: Ida, to the Hermaean crag in Lemnos; to the mighty blaze upon
the island succeeded, third, [285] the summit of Athos sacred to Zeus; and, soaring high 
aloft so as to leap across the sea, the flame, travelling joyously onward in its strength
* the pinewood torch, its golden-beamed light, as another sun, passing the message on to 
the watchtowers of Macistus. [290] He, delaying not nor carelessly overcome by sleep, did not 
neglect his part as messenger. Far over Euripus' stream came the beacon-light and signalled to 
the watchmen on Messapion. They, kindling a heap of [295] withered heather, lit up their answering 
blaze and sped the message on. The flame, now gathering strength and in no way dimmed, like a 
radiant moon overleaped the plain of Asopus to Cithaeron's ridges, and roused another relay of 
missive fire. [300] Nor did the warders there disdain the far-flung light, but made a blaze 
higher than their commands. Across Gorgopus' water shot the light, reached the mount of Aegiplanctus, 
and urged the ordinance of fire to make no delay. [305] Kindling high with unstinted force a mighty 
beard of flame, they sped it forward so that, as it blazed, it passed even the headland that looks 
upon the Saronic gulf; until it swooped down when it reached the lookout, near to our city, upon the 
peak of Arachnaeus; and [310] next upon this roof of the Atreidae it leapt, this very fire not undescended from the Idaean flame.

Such are the torch-bearers I have arranged, completing the course in succession one to the other; 
and the victor is he who ran both first and last.1 [315] This is the kind of proof and token I give you, 
the message of my husband from Troy to me.
""")

ents = [(e.text, e.label_) for e in doc.ents if e.label_ not in ("CARDINAL", "ORDINAL")] 

for ent in ents:
    print(ent)

('Hephaestus', 'ORG')
('Ida', 'PERSON')
('Hermaean', 'PRODUCT')
('Lemnos', 'ORG')
('Athos', 'ORG')
('Zeus', 'PERSON')
('Macistus', 'ORG')
('Euripus', 'ORG')
('Asopus', 'ORG')
('Cithaeron', 'ORG')
('Gorgopus', 'PERSON')
('Aegiplanctus', 'ORG')
('Saronic gulf', 'LOC')
('Arachnaeus', 'GPE')
('Idaean', 'NORP')
('Troy', 'PERSON')


## Create CSV
Now it's time to create a CSV with these locations. Unfortunately, because the model is not really designed for classical texts, we can only use this as a starting point; we need to go back through the text and locate some tokens which it missed, as well as exlude tokens which do not refer to a location. 